<a href="https://colab.research.google.com/github/Asmina-hub/AGI_Project/blob/main/ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch gcsfs
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import gcsfs
import matplotlib.pyplot as plt
from torch import nn
from torch.nn import functional as F
from torch.optim import Adam
from tqdm import tqdm
!pip install wandb
import wandb
wandb.login()
import pprint
import torchvision
from torchvision import datasets, models, transforms

import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: asminanassar77 (asminanassar77-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
#sweep config is for hyperparameter tuning here we can add just configuration
wandb.init(project='flower image classification with RESNET', config={
    "epoch": 50,
    "batch_size":16 ,
    "architecture": 'RESNET',
    "input_size" : 224,
    "pretrained" : True,
    "learning_rate" : 0.001,
    "rootfolder" : "/kaggle/input/5-flower-types-classification-dataset/flower_images"
})
config = wandb.config

In [ ]:
from google.colab import auth
import kagglehub
import os

# Download latest version
path = kagglehub.dataset_download("kausthubkannan/5-flower-types-classification-dataset")

print("Path to dataset files:", path)


Using Colab cache for faster access to the '5-flower-types-classification-dataset' dataset.
Path to dataset files: /kaggle/input/5-flower-types-classification-dataset


In [ ]:
def train_test_split(root_folder,train_size=0.8, test_size=0.2):
  train_data, test_data = [], []
  labels =os.listdir(root_folder)
  class_to_idx = {class_name: i for i, class_name in enumerate(labels)} # Create class to index mapping
  print(labels)
  for class_name in labels:
    print(class_name)
    data_path=os.path.join(root_folder,class_name)
    image_files=os.listdir(data_path)
    #len(image_files) # This line is not needed
    current_train_count=int(len(image_files)*train_size)
    current_test_count=int(len(image_files)*test_size)
    print(current_train_count)
    print(current_test_count)
    train_files=image_files[:current_train_count]
    test_files=image_files[current_train_count:]
    print(train_files)
    print(test_files)
    train_data.extend([(os.path.join(root_folder,class_name,img), class_to_idx[class_name]) for img in train_files]) # Store index
    test_data.extend([(os.path.join(root_folder,class_name,img), class_to_idx[class_name]) for img in test_files]) # Store index

  return train_data, test_data

In [ ]:
class FlowerDataLoader(Dataset): # Inherit from Dataset
  def __init__(self, data, transform):
    self.data = data
    self.transform = transform
  def __len__(self):
        return len(self.data)
  def __getitem__(self,idx):
    imagefile, label_index = self.data[idx] # label is already an index
    image = Image.open(imagefile).convert('RGB')
    if self.transform:
        image = self.transform(image)
    return image, label_index # Return the image tensor and the numerical label index

In [ ]:
train_data,test_data = train_test_split(config.rootfolder)
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
train_dataset = FlowerDataLoader(train_data, train_transform)
test_dataset = FlowerDataLoader(test_data, test_transform)

train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=config.batch_size, shuffle=False)


['Orchid', 'Sunflower', 'Tulip', 'Lotus', 'Lilly']
Orchid
800
200
['489cef8f04.jpg', '1d043ac951.jpg', '4587dffd03.jpg', '9b6ae836b2.jpg', 'a59e4c247e.jpg', '265024ff3c.jpg', '2eaca1c694.jpg', '002c100a56.jpg', '31a66409b8.jpg', 'd3bac8bbfa.jpg', '563adfe066.jpg', '8e54604127.jpg', '58927b8ce2.jpg', '2e2d816ec1.jpg', '4a56cc2ec2.jpg', 'ff3e9f1ddf.jpg', 'fb7586f725.jpg', '94340e9856.jpg', '4814b94f41.jpg', 'c676b44a48.jpg', 'e2b20074e6.jpg', 'e00cebb570.jpg', '8e817c44f6.jpg', '9a3b615050.jpg', '77213f743b.jpg', 'bacb60f2a8.jpg', 'f9415bf67e.jpg', 'b0a5a0bf87.jpg', 'bcfa11134b.jpg', '8c343b4f73.jpg', '8faa52095d.jpg', '51389fd4c0.jpg', 'd0c5c833bb.jpg', '8f3dddea3a.jpg', 'cf379c6f60.jpg', '913c4fe0ec.jpg', 'a962257c89.jpg', '8c367f73a8.jpg', '715644501e.jpg', '807f549b3e.jpg', '67ff736bc2.jpg', 'c05e25b77a.jpg', 'd6c2688fb3.jpg', '212225e213.jpg', '249ef87a3d.jpg', '0c18b0c629.jpg', '9eca1309b5.jpg', '9bc76d6fd2.jpg', '8b21d6ce81.jpg', '26463733c3.jpg', '3443efaba6.jpg', '90de27db49.jpg

In [ ]:
from torchvision.models import ResNet50_Weights
model= models.resnet50(weights=ResNet50_Weights.DEFAULT)
model.fc = nn.Linear(model.fc.in_features, 5)

for params in model.parameters():
    params.requires_grad = False


for param in model.fc.parameters():
    param.requires_grad = True

# Move model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Watch the model's weights and gradients
wandb.watch(model, log="all", log_freq=10)


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)

In [ ]:
import torch
from torchvision import models

# Load the GoogLeNet model
resnetmodel = models.resnet50(weights='DEFAULT')

print("----------- resnetmodel Architecture -----------")
print(resnetmodel)


----------- resnetmodel Architecture -----------
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0)

In [ ]:
def train_model(model, criterion, optimizer, train_loader, val_loader, epochs=10):
    for epoch in range(epochs):
        model.train()
        train_correct = 0
        train_total = 0
        running_loss = 0.0

        print(f"\nEpoch {epoch + 1}/{epochs}")
        print("-" * 30)

        for i, (images, labels) in enumerate(train_loader):

            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            batch_correct = (preds == labels).sum().item()
            train_correct += batch_correct
            train_total += labels.size(0)

            # Print every 10 batches
            if (i + 1) % 10 == 0:
                batch_acc = batch_correct / labels.size(0)
                print(f"[Batch {i+1}/{len(train_loader)}] Loss: {loss.item():.4f}, Batch Acc: {batch_acc:.4f}")

        train_acc = train_correct / train_total
        wandb.log({"epoch": epoch + 1, "train_loss": running_loss, "train_accuracy": train_acc})
        print(f"Epoch {epoch+1} Summary - Loss: {running_loss:.4f}, Train Accuracy: {train_acc:.4f}")

        # Validation
        model.eval()
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, preds = torch.max(outputs, 1)
                val_correct += (preds == labels).sum().item()
                val_total += labels.size(0)

        val_acc = val_correct / val_total
        wandb.log({"epoch": epoch + 1, "val_accuracy": val_acc})
        print(f"Validation Accuracy: {val_acc:.4f}")


In [ ]:
train_model(model, criterion, optimizer, train_loader, test_loader, epochs=config.epoch)


Epoch 1/50
------------------------------
[Batch 10/250] Loss: 1.4942, Batch Acc: 0.4375
[Batch 20/250] Loss: 1.0995, Batch Acc: 0.8750
[Batch 30/250] Loss: 1.0859, Batch Acc: 0.6875
[Batch 40/250] Loss: 1.2471, Batch Acc: 0.6250
[Batch 50/250] Loss: 0.9118, Batch Acc: 0.8750
[Batch 60/250] Loss: 0.9229, Batch Acc: 0.8125
[Batch 70/250] Loss: 0.7850, Batch Acc: 0.8750
[Batch 80/250] Loss: 0.5935, Batch Acc: 0.9375
[Batch 90/250] Loss: 0.8800, Batch Acc: 0.7500
[Batch 100/250] Loss: 0.6793, Batch Acc: 0.9375
[Batch 110/250] Loss: 0.5296, Batch Acc: 0.9375
[Batch 120/250] Loss: 0.8504, Batch Acc: 0.6250
[Batch 130/250] Loss: 0.6699, Batch Acc: 0.8125
[Batch 140/250] Loss: 0.6798, Batch Acc: 0.8125
[Batch 150/250] Loss: 0.7507, Batch Acc: 0.7500
[Batch 160/250] Loss: 0.8530, Batch Acc: 0.6250
[Batch 170/250] Loss: 0.5477, Batch Acc: 0.8750
[Batch 180/250] Loss: 0.7523, Batch Acc: 0.7500
[Batch 190/250] Loss: 0.4945, Batch Acc: 0.8750
[Batch 200/250] Loss: 0.6424, Batch Acc: 0.8125
[Batch